# HAIM test

# Test

In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
# Load filter data
df_filter = pd.read_parquet("/data/wolf6245/src/mm_study/data/f_modelling/03_model_input/data-2024-12-19-01-23-23/(3) Chronic ischaemic heart disease/y_fusion_label_not_gt.parquet")
df_folds = pd.read_pickle("/data/wolf6245/src/mm_study/data/f_modelling/03_model_input/data-2024-12-19-01-23-23/(3) Chronic ischaemic heart disease/train_test_vali_folds_fusion_label.pkl")
subject_ids_to_use = [int(i) for i in df_filter['subject_id'].unique()]
hadm_ids_to_use = [int(i) for i in df_filter['hadm_id'].unique()]

# Load data
haim_mimiciv_key_ids = pd.read_csv("/data/wolf6245/src/HAIM/data/haim_mimiciv/haim_mimiciv_key_ids.csv")
mimic_cxr_metadata = pd.read_csv("/data/wolf6245/src/HAIM/data/haim_mimiciv/mimic-cxr-2.0.0-metadata.csv")
core = pd.read_csv("/data/wolf6245/src/HAIM/data/haim_mimiciv/core/core.csv")
pickle_aux = pd.read_pickle("/data/wolf6245/src/HAIM/data/haim_mimiciv/pickle/00000000.pkl")
pickle_files = os.listdir("/data/wolf6245/src/HAIM/data/haim_mimiciv/pickle/")

# Shapes
print(f"Shape haim_mimiciv_key_ids: {haim_mimiciv_key_ids.shape}")
print(f"Shape mimic_cxr_metadata: {mimic_cxr_metadata.shape}")
print(f"Shape core: {core.shape}")
print(f"pickle files: {len(pickle_files)}")

# Filter
haim_mimiciv_key_ids_filtered = haim_mimiciv_key_ids[haim_mimiciv_key_ids['hadm_id'].astype(int).isin(hadm_ids_to_use)]
print(f"Shape haim_mimiciv_key_ids_filtered: {haim_mimiciv_key_ids_filtered.shape}")
core_filtered = core[core['hadm_id'].astype(int).isin(hadm_ids_to_use)]
print(f"Shape core_filtered: {core_filtered.shape}")
mimic_cxr_metadata_filtered = mimic_cxr_metadata[mimic_cxr_metadata['subject_id'].astype(int).isin(subject_ids_to_use)]
print(f"Shape mimic_cxr_metadata_filtered: {mimic_cxr_metadata_filtered.shape}")

# Filter cxr
core_filtered['dischtime'] = pd.to_datetime(core_filtered['dischtime'])
mimic_cxr_metadata_filtered['cxrtime'] = pd.to_datetime(mimic_cxr_metadata_filtered['cxrtime'])
max_dischtime = core_filtered.copy().groupby('subject_id')['dischtime'].max().reset_index()
max_dischtime.rename(columns={'dischtime': 'max_dischtime'}, inplace=True)
merged = mimic_cxr_metadata_filtered.copy().merge(max_dischtime, on='subject_id', how='left')
mimic_cxr_metadata_filtered_filtered = merged.copy()[merged['cxrtime'] <= merged['max_dischtime']].drop(columns=['max_dischtime'])
print(f"Shape mimic_cxr_metadata_filtered_filtered: {mimic_cxr_metadata_filtered_filtered.shape}")

# Get all hadm_ids
files_to_remove = []
for pickle_file in tqdm(pickle_files[:]):
    pickle_file_path = os.path.join("/data/wolf6245/src/HAIM/data/haim_mimiciv/pickle/", pickle_file)
    pickle_df = pd.read_pickle(pickle_file_path)
    hadm_ids_aux = [int(i) for i in pickle_df.admissions.hadm_id.unique()]
    subject_ids_aux = [int(i) for i in pickle_df.admissions.subject_id.unique()]
    if any([hadm_id not in hadm_ids_to_use for hadm_id in hadm_ids_aux]):
            files_to_remove.append(pickle_file)
print(f"Number of pickle files with hadm_ids not in filter: {len(files_to_remove)} of {len(pickle_files)}")

if False:
    # Save files back
    haim_mimiciv_key_ids_filtered.to_csv("/data/wolf6245/src/HAIM/data/haim_mimiciv/haim_mimiciv_key_ids.csv", index=False)
    mimic_cxr_metadata_filtered_filtered.to_csv("/data/wolf6245/src/HAIM/data/haim_mimiciv/mimic-cxr-2.0.0-metadata.csv", index=False)
    core_filtered.to_csv("/data/wolf6245/src/HAIM/data/haim_mimiciv/core/core.csv", index=False)

    # Delete files in files_to_remove
    for file in tqdm(files_to_remove, desc="Deleting files"):
        file_path = os.path.join("/data/wolf6245/src/HAIM/data/haim_mimiciv/pickle/", file)
        os.remove(file_path)

Shape haim_mimiciv_key_ids: (8688, 3)
Shape mimic_cxr_metadata: (377110, 16)
Shape core: (2004607, 27)
pickle files: 8688
Shape haim_mimiciv_key_ids_filtered: (6148, 3)


/data/wolf6245/envs/miniconda3/envs/haim/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/data/wolf6245/envs/miniconda3/envs/haim/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Shape core_filtered: (31381, 27)
Shape mimic_cxr_metadata_filtered: (62590, 16)
Shape mimic_cxr_metadata_filtered_filtered: (45882, 16)


100%|██████████| 8688/8688 [06:58<00:00, 20.77it/s]

Number of pickle files with hadm_ids not in filter: 2540 of 8688
